<a href="https://colab.research.google.com/github/marcelodaros/Escolha_Sua_Trilha/blob/main/ImersaoAluraIA_Projeto_Final_Escolha_sua_Trilha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalações do Google Generative AI e do PyMuPDF, para ler PDFs.

In [86]:
!pip install -q -U google-generativeai

In [1]:
!pip install -q -U PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 24.3 MB/s eta 0:00:00


Conecta no seu GoogleDrive para acessar os PDFs de base.

In [2]:
# Conectando com Google Drive para acessar as notas a serem classificadas
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Configurando o Modelo**

In [3]:
# Importacao das bibliotecas que iremos usar
from pathlib import Path
import hashlib
import google.generativeai as genai
import fitz
import os

# Configuracao da sua API_KEY do Google, acesse o menu do lado para
# adicionar ela de forma segura

from google.colab import userdata
genai.configure(api_key=userdata.get('API_KEY'))

# Configurando o modelo da AI que iremos utilizar
generation_config = {
  "temperature": 0.5,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


Fazendo a extração do conteúdo dos PDFs base para nosso ChatBot

In [4]:
# Funcao para extrair o texto das paginas dos arquivos PDF.

def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]

  # Abre o arquivo utilizando o PyMuPDF (fitz)
  with fitz.open(pathname) as doc:
    pages = []
    for page in doc:
      text = page.get_text()
      pages.append(text)

  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)
  return parts

# Criamos os prompts iniciais, que serão a base do nosso ChatBot
# troque o path_in_drive pelo caminho que está no seu Drive os arquivos
# se tiver dúvidas, acesse o vídeo na página do GitHub do projeto.
prompt_parts = [
  extract_pdf_pages("/path_in_drive/manual_estruturacao.pdf"),
  extract_pdf_pages("/path_in_drive/trilhas_do_brasil.pdf"),
  "\nBaseado nesses dois documentos, aja como um agente de turismo, pronto para tirar as dúvidas do cliente sobre as trilhas de longo alcance que temos no Brasil.",
]

In [5]:
# Inicializa o Chat com nosso histórico
chat = model.start_chat(history=[
    {
        "role": "user",
        "parts": prompt_parts[0]
    },
    {
        "role": "user",
        "parts": prompt_parts[1]
    },
    {
        "role": "user",
        "parts": prompt_parts[2]
    },
])

# Utilizando o ChatBot
Lembre-se de rodar todas as linhas anteriores, para que o ChatBot funcione
de forma correta.

In [6]:
# Importacao de algumas bibliotecas para deixar o texto mais bonito
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Iniciando o Chat com uma mensagem explicativa sobre o que ele faz
response = chat.send_message("Crie uma introdução simples, explicando que você vai ajudar a pessoa a escolher um trilha. Avise que quando ela quiser terminar o chat, basta escrever FIM,")
display(to_markdown(response.text))
prompt = input("Envie sua mensagem: ")

# Loop de utilizacao do ChatBot
while prompt != "FIM":
  response = chat.send_message(prompt)
  display(to_markdown(f"{response.text}"))
  print('-------------------------------------')
  prompt = input("Envie sua mensagem: ")

> Olá! É com muita alegria que te recebo aqui para te ajudar a desvendar as maravilhas das trilhas de longo curso do Brasil. 🧭🌳
> 
> Se você busca aventura, contato com a natureza e paisagens de tirar o fôlego, está no lugar certo! Me conte sobre seus desejos, tempo disponível, nível de experiência e qualquer preferência que tenha, como bioma, modalidade (caminhada, ciclismo, etc.) ou tipo de experiência que busca. 
> 
> Juntos, vamos encontrar a trilha perfeita para você! 😉
> 
> **Quando quiser encerrar o chat, basta digitar FIM.**
> 
> Aguardo ansiosamente para te ajudar a planejar sua próxima grande aventura! 😄


Envie sua mensagem: Estou em Minas Gerais, me indique algumas trilhas que posso fazer por aqui.


> Minas Gerais, que beleza! 🤩 É um estado riquíssimo em trilhas de longo curso, com opções para todos os gostos e níveis de experiência. Para te indicar as melhores trilhas para você, preciso entender melhor seu perfil. Me conte:
> 
> 1. **Qual o seu nível de experiência em trilhas?** (Iniciante, intermediário, avançado)
> 2. **Quanto tempo você tem disponível para a aventura?** (Dias, semanas)
> 3. **Prefere trilhas mais desafiadoras, com subidas e paisagens montanhosas, ou algo mais tranquilo?**
> 4. **Tem interesse em algum bioma específico?** Minas Gerais tem trilhas que passam pela Mata Atlântica e pelo Cerrado.
> 5. **Qual modalidade te interessa mais?** (Caminhada, ciclismo, cavalgada)
> 
> Com essas informações, posso te guiar pelas trilhas mineiras e te ajudar a escolher a que mais combina com você! 😉 


-------------------------------------
Envie sua mensagem: Sou iniciante, tenho 2 dias e prefiro trilhas mais tranquilas, que sejam no cerrado e que seja por caminhada.


> Ótimo! 😄 Minas Gerais tem opções perfeitas para iniciantes que buscam trilhas tranquilas no Cerrado. Considerando que você tem 2 dias disponíveis, recomendo dois trechos do **Caminho do Sertão**, uma trilha que te leva ao universo do escritor Guimarães Rosa:
> 
> **1. Trecho de Sagarana a Urucuia:** 
> * Distância: 24km (pode ser dividido em 2 dias, com pernoite em Urucuia).
> * Dificuldade: Moderada (alguns trechos com subidas leves).
> * Belezas: Caminho margeando o Rio Urucuia, paisagens típicas do Cerrado, visita a comunidades locais e imersão na cultura sertaneja. 
> 
> **2. Trecho de Urucuia a Riachinho:**
> * Distância: 28km (também pode ser dividido em 2 dias, com pernoite em Riachinho).
> * Dificuldade: Moderada (similar ao trecho anterior).
> * Belezas: Passagem por fazendas históricas, travessia de riachos, paisagens bucólicas e contato com a culinária local.
> 
> Ambos os trechos oferecem boa estrutura de apoio, com pousadas e restaurantes para sua comodidade. 
> 
> **Outras informações importantes:**
> 
> * Época ideal: Evite os meses mais chuvosos (dezembro a março).
> * O que levar: Roupas confortáveis, calçados adequados para trilha, protetor solar, repelente, chapéu, água e lanches.
> 
> Lembre-se: mesmo em trilhas mais tranquilas, é fundamental se preparar e se informar sobre as condições do percurso. Entre em contato com os gestores da trilha para confirmar as informações e garantir uma experiência segura e inesquecível! 😉
> 
> Gostaria de saber mais sobre algum desses trechos? Ou prefere que eu te apresente outras opções? 😊 


-------------------------------------
Envie sua mensagem: Quero a primeira opção


> Excelente escolha! 😊 O trecho de Sagarana a Urucuia é realmente encantador e perfeito para uma imersão suave no universo do sertão mineiro. 
> 
> Vou te passar algumas dicas para você planejar essa aventura:
> 
> **Informações importantes:**
> 
> * **Contato:**  Recomendo entrar em contato com o Instituto Rosa e Sertão, responsável pela gestão do Caminho do Sertão. Eles podem te passar informações atualizadas sobre a trilha, condições do percurso, estrutura de apoio e dicas de segurança. O email deles é: caminhodosertao@gmail.com;  rosaceasinstituto@gmail.com
> * **Como chegar:** Partindo de Brasília, você pode pegar um ônibus para Arinos (MG). De lá, é possível contratar um transporte local para te levar até Sagarana, ponto de partida da trilha. 
> * **Hospedagem:** Em Urucuia, você encontra pousadas charmosas e aconchegantes, com opções para todos os bolsos. 
> * **Alimentação:** A culinária local é rica em sabores e aromas do Cerrado. Aproveite para experimentar pratos típicos, como o frango com pequi e a carne de sol com mandioca.
> * **Atrativos:** Além das belezas naturais do percurso, você pode visitar a casa onde Guimarães Rosa passou parte da sua infância em Urucuia, hoje transformada em museu. 
> 
> **Dicas extras:**
> 
> * Leve um bom livro de Guimarães Rosa para ler durante a trilha e se inspirar ainda mais com a paisagem.
> * Converse com os moradores locais! Eles são os melhores guias para te apresentar as histórias, lendas e costumes do sertão.
> * Registre cada momento! A beleza do Cerrado rende fotos incríveis.
> 
> Tenho certeza de que você vai amar essa experiência! 😊
> 
> Deseja saber mais alguma coisa sobre esse trecho? Ou quer que eu te ajude com algum outro aspecto da sua viagem? 😄 


-------------------------------------
Envie sua mensagem: Não estou em Brasília, estou em Belo Horizonte, qual a melhor forma de chegar em Sagarana?


> Ah, entendi! 😊 Para chegar em Sagarana partindo de Belo Horizonte, a melhor opção é ir de ônibus até Arinos (MG). A viagem dura em torno de 8 horas e você encontra diversas empresas que fazem esse trajeto, como a Gontijo e a Pássaro Verde. 
> 
> Chegando em Arinos, você pode contratar um transporte local (táxi ou carro particular) para te levar até Sagarana, que fica a cerca de 30km de distância. 
> 
> **Dica:** Entre em contato com as pousadas de Urucuia com antecedência. Algumas delas oferecem serviço de transfer de Arinos para Sagarana e de Urucuia de volta para Arinos, o que pode facilitar bastante sua logística. 😉
> 
> Te ajudei com a sua dúvida? 😊 Tem mais alguma pergunta? 


-------------------------------------
Envie sua mensagem: FIM
